In [11]:
%matplotlib inline
import utils; reload(utils)
from utils import *

In [12]:
import os

## Train linear model on predictions

- Use a Dense() layer to convert the 1,000 predictions given by our model into a probability of dog vs cat
- Train a linear model to take the 1,000 predictions as input, and return dog or cat as output, learning from the Kaggle data. 

### Training a model 

#### Config steps
- Copy a small amount of our data into a 'sample' directory, with the exact same structure as our 'train' directory
- Should do all of initial testing using a dataset small enough that we don't have to wait.

In [13]:
path = "data/redux/sample/"
#path = "data/redux/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

Trial and error to find the max batch size
- The max size doesn't give an 'out of memory' error.

In [14]:
#batch_size = 100
batch_size = 4

Start with VGG16 model because we'll use its predictions and features

In [15]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

#### Our overall approach:
1. Get the true labels for every image: isCat or isDog.
2. Get the 1,000 ImageNet category predictions for every image.
3. Feed these predictions as input to a linear model.

Grab training and validation batches:

In [16]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 8 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


To avoid loading the images every time we want to use them, we should save the processed arrays.
- The fastest way to save and load numpy arrays, use bcolz. This also compresses the arrays, so we save disk space. 
- The provided functions (save_array, load_array) are defined in utils.py

'get_data' function joins the arrays from all the batches.
Use this to grab the training and validation data:

In [17]:
val_data = get_data(path+'valid')

Found 8 images belonging to 2 classes.


In [18]:
trn_data = get_data(path+'train')

Found 16 images belonging to 2 classes.


In [19]:
trn_data.shape

(16, 3, 224, 224)

#### (1) Get true labels for every image
Keras returns classes as a single column, so we convert them to one hot encoding - using onehot function:

In [23]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [24]:
trn_labels.shape

(16, 2)

In [25]:
trn_classes[:4]

array([0, 0, 0, 0], dtype=int32)

In [26]:
trn_labels[:4]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

For each image, features for our linear model include
- the label
- 1,000 imagenet probabilties from VGG16

#### (2) Get the 1,000 ImageNet category predictions for every image.

In [27]:
#Generate predictions for the input samples.
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [28]:
trn_features.shape

(16, 1000)

In [29]:
save_array(model_path + 'train_lastlater_features.bc', trn_features)
save_array(model_path + 'valid_lastlater_features.bc', val_features)

#### (3) Feed these predictions as input to a linear model.
Define our linear model:

In [30]:
# 1000 inputs (the saved features), and 2 outputs (for dog & cat)
lm = Sequential([ Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

Fit the model:

In [31]:
#batch_size=64
batch_size=4

In [32]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 0s - loss: 0.5888 - acc: 0.6250 - val_loss: 0.4288 - val_acc: 0.8750
Epoch 2/3
16/16 [==============================] - 0s - loss: 0.3033 - acc: 1.0000 - val_loss: 0.3214 - val_acc: 0.8750
Epoch 3/3
16/16 [==============================] - 0s - loss: 0.2207 - acc: 1.0000 - val_loss: 0.2494 - val_acc: 0.8750


In [33]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_4 (Dense)                  (None, 2)             2002        dense_input_1[0][0]              
Total params: 2002
____________________________________________________________________________________________________
